# **Tutorial Basic Spark RDD**

## Alif Mahardhika - 1706021934

In [19]:
'''
Karena saya menjalankan spark di machine windows pribadi, perlu ada konfigurasi imports tambahan:
'''
import findspark
SPARK_HOME='C:\spark\spark-3.0.1-bin-hadoop2.7'
findspark.init('C:/spark/spark-3.0.1-bin-hadoop2.7')
findspark.find()
import pyspark
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

## Persiapan:

Pada bagian ini sebuah spark context dan session di inisiasi. Saya juga mendefinisikan variabel header untuk menyimpan nama-nama column berdasarkan interpretasi saya.

### Soal 2.1: Load data format ke RDD dan hitung jumlah user.

In [47]:
#Saya tidak tahu kenapa tapi kalau bagian ini dihilangkan machine menganggap sudah ada spark context yang berjalan. Kalau di stop tanpa diinisiasi juga error. Jadi saya biarkan dan kemudian di stop
spark = SparkSession \
.builder \
.appName("Python Spark create RDD example") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()
spark.stop()

#inisiasi context dan session
sc= SparkContext('local','example')
spark = SparkSession(sparkContext=sc)

#membaca file u.user sebagai textFile
rdd_raw = sc.textFile("ml-100k/u.user")
header = ['no', 'age', 'sex', 'occupation','unused']

#dari textFile, setiap token di split berdasarkan karakter |
rdd = rdd_raw.map(lambda x: x.split('|'))
rdd.take(2) #print dua row teratas untuk memastikan sudah terbaca

[['1', '24', 'M', 'technician', '85711'], ['2', '53', 'F', 'other', '94043']]

In [49]:
from pyspark.sql import Row

# Fungsi untuk memformat ke row object RDD
def list_to_row(keys, values):
    row_dict = dict(zip(keys, values))
    return Row(**row_dict)

# Format data menjadi format RDD, dan print jumlah user
rdd_rows = rdd.map(lambda x: list_to_row(header, x))
print("Number of users: " +str(rdd_rows.count()))



Number of users: 943


### Soal 2.2: Buat fungsi map yang membagi user ke rentang usia 10-80 tahun buat bin sebesar 10 tahun setiap kategori.

In [69]:
df = rdd.toDF(header)
aged_80_less = df.filter("age <=80")
between_10_80 = aged_80_less.filter("age>10")

over_age = df.filter("age > 80")
under_age = df.filter("age < 10")

between_10_80.count()

941

In [79]:
import pyspark.sql.functions as F

a =df.withColumn('age_group',
F.when((df.age>=0)&(df.age<=10),'under_age')\
.when((df.age>10)&(df.age<=80),'relevant_age')\
.when((df.age>80),'over_age'))\
.drop_duplicates(subset=['age_group']).show()

TypeError: withColumn() got an unexpected keyword argument 'a'

In [78]:
# outlier_count = spark.sparkContext.accumulator(0)
df.show()

+---+---+---+-------------+------+
| no|age|sex|   occupation|unused|
+---+---+---+-------------+------+
|  1| 24|  M|   technician| 85711|
|  2| 53|  F|        other| 94043|
|  3| 23|  M|       writer| 32067|
|  4| 24|  M|   technician| 43537|
|  5| 33|  F|        other| 15213|
|  6| 42|  M|    executive| 98101|
|  7| 57|  M|administrator| 91344|
|  8| 36|  M|administrator| 05201|
|  9| 29|  M|      student| 01002|
| 10| 53|  M|       lawyer| 90703|
| 11| 39|  F|        other| 30329|
| 12| 28|  F|        other| 06405|
| 13| 47|  M|     educator| 29206|
| 14| 45|  M|    scientist| 55106|
| 15| 49|  F|     educator| 97301|
| 16| 21|  M|entertainment| 10309|
| 17| 30|  M|   programmer| 06355|
| 18| 35|  F|        other| 37212|
| 19| 40|  M|    librarian| 02138|
| 20| 42|  F|    homemaker| 95660|
+---+---+---+-------------+------+
only showing top 20 rows

